<font size=8><center> API </center></font>

Target - create an API that: 
- accepts images as single images (jpg, jpeg, PNG, TIFF), folders of images and URLs
- converts images to JPEG format
- converts JPEGs to NumPy arrays
- creates an input list of NumPy arrays for model
- preprocesses input list
- makes class predictions based on model

In [62]:
#from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os


#30Aug2023 imports
#pip install tensorflow
#pip install opencv-python
#pip install split-image

import cv2
from split_image import split_image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.models import load_model

In [2]:
img = "/Users/leila/code/meloeckert/solar_project/raw_data/Solar panel Image/Electrical-damage/"

In [14]:
def process(images : dict(filename:bytes)):
    """
    Predict presence of damage and damage class based on image
    """
    def get_image_from_bytes(filename, bytes):
        
    
    
    def convert_to_jpeg(filepath):
        #Load image
        img= Image.open(filepath)
        
        # Convert the image to RGB mode
        img = img.convert("RGB")
                
        # Save the image in JPEG format as <original filepath>.jpeg to the "jpeg_tmp" folder 
        name = os.path.basename(filepath).replace("jpg", "jpeg")
        new_filepath = os.path.join(os.getcwd(), "jpeg_tmp", name)
        img.save(new_filepath, "JPEG")
                
        # Close the image to free resources
        img.close()
        
        #print("Image format conversion complete.")     
        return new_filepath

    
    def file_to_arrays_list(filepath):
        '''Converts JPEG files to arrays and appends arrays & filenames to 'arrays' & 'filenames' lists
        Appends filenames of unprocessible files to 'invalid' list'''
        
        image= Image.open(filepath)
        image_array=np.array(image)
        arrays.append(image_array)


    filepaths = []
    arrays=[]
    invalid = []
    
    #Gets file paths for input files
    for root, dirs, files in os.walk(image_path):
        for file in files:
            filepath = os.path.join(root, file)
            #Ignores macOS hidden system file in directories
            if filepath.lower().endswith(".ds_store"):
                continue
            
            try:
                #Converts JPEG files to arrays and add filepath to 'filepaths' list
                if file.lower().endswith(".jpeg"):
                    file_to_arrays_list(filepath)
                    filepaths.append(filepath)
                
                #Processes non-JPEG image files, converts to arrays and adds original image filepath to 'filepaths' list
                else:           
                    #Creates jpeg_tmp for converted images if not already created                        
                    if not os.path.exists(os.path.join(os.getcwd(), "jpeg_tmp")):
                            os.makedirs(os.path.join(os.getcwd(), "jpeg_tmp"))
                
                    #Save JPEG to jpeg_tmp with original filepath as name              
                    original_filepath=filepath
                    filepath = convert_to_jpeg(filepath)
                    
                    #Converts JPEGs to arrays and adds original image filepath to 'filepaths' list
                    file_to_arrays_list(filepath)
                    filepaths.append(original_filepath)
                    os.remove(filepath)
            
            except Exception as e:
                if file.lower().endswith(".DS_Store"):
                    continue
                else:
                    print("Error processing:", filepath)
                    print("Error:", e)
                    invalid.append(filepath)
                    if filepath.lower().endswith(".ds_store"):
                        print("Skipping .DS_Store:", filepath)   
                        print(filepath)
                    

    print("Filepaths list is "+str(len(filepaths)))
    print("Arrays list"+str(len(arrays)))
    print("Invalid is "+str(len(invalid)))
    
    return {'Filename':filepaths, 
        'Prediction': arrays,
           'Errors': invalid}

In [55]:
def predictImage_multiclass(filepath, model):
        # Takes an image and a model
        
        img = Image.open(filepath)      
        img = img.resize((225,225))
        img.show()
        img = img_to_array(img)
        print(img.shape)
        img = img.reshape((-1, 225, 225, 3))
          
        res = model.predict(img)
        print(f"Probabilities: ")
        names_of_classes = class_names
        print(f"{names_of_classes}")
        print(f"{res[0]}")
        print(f"Result: {names_of_classes[find_index_of_max_element(res[0].tolist())]}")  
        return res 
    

In [55]:
def predictImage_multiclass(filepath, model):
        # Takes an image and a model
        tiles=[]
        img = Image.open(filepath)      
        img = img.resize((675,675))
        tiles = img.split_image(3,3)
        img = img.resize((225,225))
        img.show()
        img = img_to_array(img)
        print(img.shape)
        img = img.reshape((-1, 225, 225, 3))
          
        res = model.predict(img)
        print(f"Probabilities: ")
        names_of_classes = class_names
        print(f"{names_of_classes}")
        print(f"{res[0]}")
        print(f"Result: {names_of_classes[find_index_of_max_element(res[0].tolist())]}")  
        return res 
    

In [46]:
model = load_model('/Users/leila/code/meloeckert/solar_project/model_multi.h5')

In [43]:
cats = ["Bird-drop", "Clean", "Dusty", "Electrical-damage", "Physical-Damage", "Snow-Covered"]

file = "/Users/leila/code/meloeckert/solar_project/raw_data/API-test-data"

img_file="/Users/leila/code/meloeckert/solar_project/raw_data/Solar panel Image/Dusty/Dust (209).jpg"




In [51]:
def find_index_of_max_element(input_list):
    max_value = max(input_list)
    max_index = input_list.index(max_value)
    return max_index

In [57]:
class_names= ['clean', 'damage', 'dirt']
predictImage_multiclass(img_file, model)

(225, 225, 3)
1/1 [==============================] - 0s 29ms/step
Probabilities: 
['clean', 'damage', 'dirt']
[0.39299455 0.3835542  0.22345123]
Result: clean


array([[0.39299455, 0.3835542 , 0.22345123]], dtype=float32)

In [27]:
img_to_array(Image.open(img_file))

array([[[177., 216., 255.],
        [177., 216., 255.],
        [177., 216., 255.],
        ...,
        [180., 213., 246.],
        [187., 214., 241.],
        [193., 214., 233.]],

       [[109., 152., 194.],
        [109., 152., 194.],
        [109., 152., 194.],
        ...,
        [111., 147., 181.],
        [118., 149., 177.],
        [202., 224., 247.]],

       [[ 91., 139., 188.],
        [ 91., 139., 188.],
        [ 91., 139., 188.],
        ...,
        [ 92., 134., 174.],
        [100., 136., 170.],
        [200., 227., 255.]],

       ...,

       [[ 31.,  29.,  30.],
        [ 29.,  27.,  28.],
        [ 27.,  25.,  26.],
        ...,
        [179., 165., 154.],
        [178., 164., 153.],
        [160., 146., 137.]],

       [[ 28.,  26.,  27.],
        [ 25.,  23.,  24.],
        [ 21.,  19.,  20.],
        ...,
        [166., 152., 139.],
        [159., 145., 134.],
        [131., 117., 108.]],

       [[ 59.,  57.,  58.],
        [ 54.,  52.,  53.],
        [ 48.,  